In [1]:
import pandas as pd
df=pd.read_csv('finaldata.csv')
df.head()

,track_id,name of singers,type,duration,genre_top,genres,genres_all,language,listens,title,MFCC,Spectral_Centroid,Zero_Crossing_Rate
0,2,AWOL - A Way Of Life,Album,168,Hip-Hop,[21],[21],en,1293,Food,[-122.713936 117.76008 -42.334175 3...,3683.997535,0.098364
1,3,AWOL - A Way Of Life,Album,237,Hip-Hop,[21],[21],en,514,Electric Ave,[-151.68916 125.360504 -37.11494 4...,3353.226098,0.088935
2,5,AWOL - A Way Of Life,Album,206,Hip-Hop,[21],[21],en,1151,This World,[-1.62232422e+02 1.31435989e+02 -1.84267788e+...,2928.398703,0.059334
3,10,Constant Hitmaker,Album,161,Pop,[10],[10],en,50135,Freeway,[-75.63154 154.53761 -63.849487 22.50...,2786.793350,0.080989
4,20,Niris,Album,311,Rock,"[76, 103]","[17, 10, 76, 103]",en,361,Spiritual Level,[-1.09059998e+02 1.18665329e+02 3.29060984e+...,3341.462873,0.052937


In [2]:

# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Print the count of missing values for each column
print("Missing Values:")
print(missing_values)


Missing Values:
track_id              0
name of singers       0
type                  0
duration              0
genre_top             0
genres                0
genres_all            0
language              0
listens               0
title                 0
MFCC                  0
Spectral_Centroid     0
Zero_Crossing_Rate    0
dtype: int64


In [3]:

# Assuming 'genre_top' is your target variable
class_distribution = df['MFCC'].value_counts()

print("Class Distribution:")
print(class_distribution)

# Calculate the proportion of each class
class_proportion = class_distribution / class_distribution.sum()

print("\nClass Proportion:")
print(class_proportion)


Class Distribution:
[-1131.371     0.        0.        0.        0.        0.        0.\n     0.        0.        0.        0.        0.        0.        0.\n     0.        0.        0.        0.        0.        0.   ]                                                                                                                  39
[-250.64679      99.93232     -56.840397     20.675358    -41.70628\n   -6.9831963   -25.66879       6.72321      -9.777294      4.9054203\n   -1.490654     -0.91331685  -11.228164     -0.33084515   -9.596461\n   -1.1122949    -3.5145388     7.3502083    -6.7539988     5.1603227 ]                                11
[-361.16373     168.41776      22.06176     -21.597418     -9.362466\n  -10.066576    -13.307745     -2.128851      7.8589582    11.549643\n   13.717208     11.56835       3.788877     -6.316524    -11.787543\n   -8.333406      0.57650965    6.0143986     4.6708713     3.1628852 ]                                11
[-335.43845    100.95326      2.

In [4]:

import numpy as np

# Convert MFCC column to float type
df['MFCC'] = df['MFCC'].apply(lambda x: np.array(x.strip('[]').split()).astype(np.float32))


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into features (X) and target (y)
X = df['MFCC'].tolist()  # Convert MFCC column to list
y = df['genre_top']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict the classes for the testing set
y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.67825760067666


In [6]:
import pickle

# Train your model (already done)
# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# Save the trained model to a file
model_path = 'random_forest_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(rf_classifier, f)


In [12]:
import pickle
import librosa
import numpy as np

# Function to load the trained model
def load_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

# Function to extract features from an audio file
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    return np.mean(mfcc, axis=1)

# Function to predict genre label for an audio file
def predict_genre(model, file_path):
    # Extract MFCC features from the audio file
    features = extract_features(file_path)

    # Predict the genre label
    predicted_label = model.predict([features])[0]
    return predicted_label

def main():
    # Load the trained model
    model_path = 'random_forest_model.pkl'  # Provide the path to your trained model
    model = load_model(model_path)

    # Input the path to the audio file
    file_path = input("Enter the path to the MP3 file: ")

    try:
        # Predict the genre label
        predicted_label = predict_genre(model, file_path)
        print("Predicted Label:", predicted_label)
    except Exception as e:
        print("Error:", e)

if __name__ == "__main__":
    main()


Predicted Label: Hip-Hop
